## Conformalized Survival Analysis with Right-Censoring

This notebook demonstrates the use of DR-COSARC on real data sets.

In [28]:
# Load required libraries
suppressMessages(library(tidyverse))

## Load the data

In [79]:
source("utils_data_preprocess.R")

for(data.name in c("VALCT", "PBC", "GBSG", "METABRIC", "COLON", "HEART", "RETINOPATHY")) {   
    cat(sprintf("Processing %s...\n", data.name))
    data <- load_data(data.name) %>% preprocess_data()
    data %>% write_csv(sprintf("../../data/data_csv/%s.csv", data.name))
}

Processing VALCT...
Processing PBC...
Processing GBSG...
Processing METABRIC...


Rows: 2509 Columns: 39
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (27): Study ID, Patient ID, Sample ID, Type of Breast Surgery, Cancer Ty...
dbl (12): Age at Diagnosis, Cohort, Neoplasm Histologic Grade, Lymph nodes e...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Processing COLON...
Processing HEART...
Processing RETINOPATHY...


In [82]:
for(data.name in c("VALCT", "PBC", "GBSG", "METABRIC", "COLON", "HEART", "RETINOPATHY")) {   
    data <- load_csv(data.name)
    cat(sprintf("Dataset: %s. n = %d, p=%d.\n", data.name, nrow(data), ncol(data)-2))
}

Dataset: VALCT. n = 137, p=6.
Dataset: PBC. n = 418, p=17.
Dataset: GBSG. n = 2232, p=6.
Dataset: METABRIC. n = 1981, p=41.
Dataset: COLON. n = 1858, p=11.
Dataset: HEART. n = 172, p=4.
Dataset: RETINOPATHY. n = 394, p=5.


In [51]:
df <- survival::colon %>%
            select(-id, -study, -etype) %>%
            select(time, status, everything())

df

,time,status,rx,sex,age,obstruct,perfor,adhere,nodes,differ,extent,surg,node4
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1521,1,Lev+5FU,1,43,0,0,0,5,2,3,0,1
2,968,1,Lev+5FU,1,43,0,0,0,5,2,3,0,1
3,3087,0,Lev+5FU,1,63,0,0,0,1,2,3,0,0
4,3087,0,Lev+5FU,1,63,0,0,0,1,2,3,0,0
5,963,1,Obs,0,71,0,0,1,7,2,2,0,1
6,542,1,Obs,0,71,0,0,1,7,2,2,0,1
7,293,1,Lev+5FU,0,66,1,0,0,6,2,3,1,1
8,245,1,Lev+5FU,0,66,1,0,0,6,2,3,1,1
9,659,1,Obs,1,69,0,0,0,22,2,3,1,1
